# Principles of Principal Component Analysis

**Abstract: Principal component analysis is the clever application of techniques from linear algebra to analysis of multivariate data in statistics. At its heart, it is nothing but the diagonalization of the covariance matrix for a sample of multivariate data, and analysis of the Eigenvalues and Eigenvectors of said matrix... but this explanation is perhaps a little to concise to be useful.**

Let's start with an illustrative example. Take the two faces below. They are the faces of two different individuals, right? But why? Here's one possible answer, which has some flaws: for each face, we make every possible measurement--dimensions of best-fit oval encompassing face, height from chin to hairline, chin width, nasal distance, pupilary distance, length of nose, ear lobe connectivity, upper lip, etc. Each of these measurements is encoded as a number. With $N$ measurements, then, we would encode each face as a point in $\mathbb{R}^N$. These quantities that we measure are generally referred to as "features". In slightly more mathematical terminology, the encoding is a function $\phi$ from face-space (the set of all possible faces) to $\mathbb{R}^N$. Then two faces, $F_1, F_2$ are from distinct individuals if $\phi(F_1)\neq \phi(F_2)$... right? Probably not. This is neither how you distinguish individuals, nor is it very stable, in that a picture of a face from two different (even very close) angles will produce different measurements. 

So this is a bad choice for how to recognize faces. It would be rather time consuming, and you probably aren't good enough at making the precise distance measurements to capture the exact value of the function on each face. Also, the function seems ill-defined, since facial expression can change the measurements, so the same face may have many points associated to it.  Furthermore, the most distinguishing feature (or subset of features) may not actually be one of the measurements that we decided to take. Perhaps the distance from the right pupil to the left corner of the mouth is the better measurement.  Or perhaps it's a feature you've never considered, like twice your head circumference minus quintuple your pupilary distance. The point is, the elementary features we measure aren't necessarily the ones that are most convenient for discrimination. On the other hand, I doubt that you need to know the light intensity in *every* pixel in order to distinguish two faces. 

## Inputs

Take a sample from a population, and measure a collection of features from this sample. These data are stored in vectors $\{x_1,\dotsc, x_m\}\subset \mathbb{R}^N$, one for each sample, where $N$ is the number of features measured. The input to the algorithm is this list of vectors together with an integer $d<N$, the number of features you would actually like in the end. 

## Output
The algorithm then finds a collection of $d$ features, obtained as linear combinations of the original features, which capture as much of the variance in the data as possible. The idea here is that if we want to discriminate among samples, the first measurement we should make is the one that varies the most among all of the samples. If everyone has the same-sized chin, then I don't want to measure the chin, because it doesn't impart new wisdom upon me. But if there is a big spread among chin sizes, it does give me information. 

## The Math
$\newcommand{\proj}{\operatorname{proj}}$
$\newcommand{\Var}{\operatorname{Var}}$
You may already see that the above paragraph prescribes that we diagonalize the variance-covariance matrix of the sample, so let's see why that's the right intuition. Recall that if we align the vectors $x_i$ as the columns of a matrix $X$, the *variance-covariance matrix*, denoted $\Var(X)$ is the matrix $XX^T$, so $\Var(X)_{ij}=\sum_k x_{ik}x_{jk}$. If the mean of each feature is zero, this measures how much the i-th and j-th sample co-vary, so we'll always assume that we've mean-centered the data. Our goal will be to construct a new list of features $F(x_i)$ for each $i$, that is, a function $F: \mathbb{R}^N \rightarrow \mathbb{R}^d$ with some nice properties. 

- If two features, $F_1$ and $F_2$ co-vary, then we could glean *some* information about $F_2$ from $F_1$. To maximize the information we get, we should minimize co-variation, which we could do! If $F(X)$ is the matrix whose columns are $F(x_1), F(x_2),\dotsc, F(x_N)$, then we would ask that $\Var(F(X))$ be a diagonal matrix. 
- The total variance that we've captured with the function $F$ would then be the sum of the diagonal entries of the matrix $\Var(F(X))$, and we want this to be maximal.
- The procedure should be the same for each dimension $d$
- PCA is linear, so $F$ should be a linear transformation, encoded by a matrix $A$

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression

In [ ]:
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
import matplotlib.animation as animation

## The Data
The data that I'll import is the training data for the so-called MNIST dataset. The data and description of the dataset is available at Yann LeCun's webpage here: [MNIST Data](http://yann.lecun.com/exdb/mnist/). Essentially each row represents a $28\times 28$ pixel image of a handwritten digit, with the first column storing the class (which digit is it). Our goal here is not to construct a classifier, though we could. You can find the benchmarks others have achieved for this goal in the aforementioned link. With just a linear classifier, you can achieve just 12% error, which is pretty great considering random guessing would put you at a 90% error rate! Instead, we're just using this data to visualize the effects of PCA. 

For the sake of feasibility of timing, we'll use just 4000 images as our training set, and 2000 more as our "test" set. There are two ways to do this. If we just want to pick the first 4000 rows (without the label), we'd take `df.values[:4000,1:]`, which produces a $4000\times 28^2$ array of data. Because the point of this little exercise is to practice some of the commands, I'll instead use the `train_test_split` function from `sklearn.model_selection`. 

## File Formats
The data is stored in a rather interesting manner. More information can be found at the preceeding webpage, but here's some useful bullet points: 
- The training data (and test data, for that matter) have the first 16 entries encoding four 32 bit integers
- The first of these integers is the so-called "magic number". It  tells you the dype of the data as well as its shape. From Lecun's page: 
> The first two bytes are always 0. The third byte codes the type of the data:...0x09: unsigned byte... The 4-th byte codes the number of dimensions of the vector/matrix (data)

We'll see below that the pixel values are stored as unsigned bytes
- The next 32 bit integers tell us about the size of the data. The first of them is the number of images (first dimension), the next the number of rows of each image (second dimension), and the third is the number of columns of each image (third dimension). If you had data with higher structural dimension, you would have more rows. 
- Each following byte is the pixel value (in the range [0,256), with 0 indicating white).
- The label files are similar, but they only have one dimension--the label.

In [ ]:
train_images='MNIST_data/t10k-images-idx3-ubyte'
train_labels='MNIST_data/t10k-labels-idx1-ubyte'

Below, I've included an illustration of how to pull this data from these files, because it took me some learning to get there as well. 

In [ ]:
Images=np.zeros((10000,28,28))
with open(train_images,'rb') as f:
    f.read(16)
    for n in range(10000):
        for j in range(28):
            for i in range(28):
                Images[n,j,i]=ord(f.read(1))
Labels=np.zeros(10000)
with open(train_labels,'rb') as l:
    l.read(8)
    for n in range(10000):
        Labels[n]=ord(l.read(1))


Now we have the images stored in the array "Images". I intentionally stored them as $28\times 28$ pixel images, but for PCA you want vectors, so let's reshape the images as $784$-dimensional vectors.

In [ ]:
Data=Images.reshape((10000,-1)) #the new shape is inferred

This is swell. We have the data, and if we had a lot of time, we'd PCA this whole thing. Since we're just trying to illustrate some things, let's chop down the size of the data to only work with 4,000 images. 

In [ ]:
X_train=Data[:4000,:]
y_train=Labels[:4000]
print("X_train shape: %s, y_train shape: %s" %(X_train.shape, y_train.shape))

In [ ]:
images_and_labels=list(zip(Images[:4000,:,:],y_train))

Let's see some of these digits. We'll plot about 100 of them to see some of the rough sketching that will be available. Conveniently, the `matplotlib` plotting library will let you show images in panels. A few useful tools here: 
- `plt.subplots(m,n)` allows you to produce a figure object (here 'f'), and an Axes object (here 'axarr') that has cells
- The cells of 'axarr' are indexed like the entries in a matrix, and these cells can be treated like figures themselves, so a plot can be produced in each cell separately

In [ ]:
f,axarr=plt.subplots(10,10,sharey=True,figsize=(40,40))
for index, (image,label) in enumerate(images_and_labels[:100]):
    N=int(index/10)
    i=index%10
    axarr[N,i].imshow(image)
    axarr[N,i].axis('off')
    axarr[N,i].set_title('Digit %i' % (label))
plt.savefig('digits.pdf')

## Time to learn
Though we could easily code up PCA using Scipy's Linear Algebra library (as you saw above, it all boils down to finding Eigenvalues), we can instead import the PCA functionality from a package called Scikit Learn. It's the machine learning extension of Scipy. I'm going to create a PCA analyzer to produce 150 principal components. Why 150? Well it's less than 750. 

In [ ]:
pca=PCA(n_components=700)

Here, we run the PCA analysis on the 4000 images in val, and store the 150 principal components in the variable cpts. The object pca is a class, and it has many methods. One is .fit(input), which will ask the analyzer to run the algorithm and pick number of components (`n_components`) desired. It stores an internal variable .components_ which contains the list of the components as (row) vectors. We're going to take each principal component (which, recall, is an image) and resize it so that we can visualize it. In the cell following, we print a selection from the first 44 Principal Component images.

Recall that in PCA, each image will be compared to the Principal Component images via the dot product. A large dot product means high similarity, and a small dot product means very little contribution from the principal component. 

In [ ]:
pca.fit(X_train) #Our little PCA model going to school 
cpts=pca.components_ #Now we ask what it learned
print(cpts.shape)
eig_images=np.reshape(cpts,(700,28,28)) #And reshape that to actual images

In [ ]:
f,axarr=plt.subplots(5,5,sharey=True,figsize=(30,30))
for N in range(5):
    n=10*N
    for i in range(5):
        axarr[N,i].imshow(eig_images[n+i])
        axarr[N,i].axis('off')
        axarr[N,i].set_title('EigenImage %i' %(n+i))
plt.savefig('Eigenimages.pdf')

Next we look at how the compressed data looks. Unfortunately, while the PCA object has methods `.transform(X)` (which gives the new coordinates of the data $X$, computed via taking the dot product/projection of each row onto each Eigenimage) and `.inverse_transform(X')` (which takes the compressed data and embeds it back into the full image space), it won't give you partial reconstruction, which is what I'd like to do below. We want to look at how clearly the image is reconstructed based on how many principal components we use. In a slight hack, I'll do the following: 
- transform/compress the training data using the `.transform` method
- apply the inverse transform to slices of the compressed data

In [ ]:
X_compressed=pca.transform(X_train) #np.dot(np.dot(val,pca.components_.transpose()),pca.components_)
X_decompressed=pca.inverse_transform(X_compressed)
Images_compressed=np.reshape(X_decompressed,(4000,28,28))
images_and_labels_compressed=list(zip(Images_compressed, y_train))

In [ ]:
print('Principal component shape: %s, Compressed data shape: %s' %(cpts.shape,X_compressed.shape))

In this next cell, we'll create a generator to take care of what I see as the shortcoming of the PCA class in Scikit-learn. When you pass the previously-compressed data to this generator together with the principal components, it builds a generator whose $n$th step returns the data as reconstructed from $n$ of the principal components. 

In [ ]:
def data_reconstruction(data,principal_components,by_n=1):
    n,c=data.shape
    C,N=principal_components.shape
    M=c//by_n
    assert C==c
    L=np.empty((n,N))
    m=0
    while m+by_n<c:
        iteration=m
        A=data[:,m:m+by_n]
        B=principal_components[m:m+by_n,:]
        L+=np.dot(data[:,m:m+by_n],principal_components[m:m+by_n,:])
        m+=by_n
        yield L,iteration
    yield L+np.dot(data[:,m:c],principal_components[m:c,:]),m


In [ ]:
X_compression_layers_generator=data_reconstruction(X_compressed,cpts,1)  
f,axarr=plt.subplots(27,25,sharey=True,figsize=(30,30))
for N in range(27):
    for i in range(25):
        image_array,num_layers=next(X_compression_layers_generator)
        axarr[N,i].imshow(image_array.reshape((4000,28,28))[4,:,:])
        axarr[N,i].axis('off')
        axarr[N,i].set_title('Reconstructed by %s layers' %(num_layers))
plt.savefig('Reconstruction.pdf')

In [ ]:
X_compression_layers_generator=data_reconstruction(X_compressed,cpts,1)  
fig=plt.figure()
im_one=next(X_compression_layers_generator)[0].reshape((4000,28,28))[4,:,:]
im=plt.imshow(im_one)#,cmap='Greys')
im.set_array(im_one)
plt.xlabel(('Using %d components' %(num_layers)))
def updatefig(*args):
    image_array,num_layers=next(X_compression_layers_generator)
    im.set_array(image_array.reshape((4000,28,28))[4,:,:])
    plt.xlabel(('Using %d components' % (num_layers)))
    return im,
Writer=animation.writers['ffmpeg']
writer=Writer(fps=15,metadata=dict(artist='Andrev'),bitrate=1800)
ani=animation.FuncAnimation(fig,updatefig,interval=5,blit=True)
ani.save('test_animation.mp4',writer=writer)
#def updatefig(*args):
    

In [ ]:
#X_compression_layers=[np.dot(np.dot(X_train,cpts[:i,:].transpose()),
#                            cpts[:i,:]).reshape((4000,28,28)) for i in range(150)]

#X_compression_layers=[pca.inverse_transform(X_compressed[])]

In [ ]:
f,axarr=plt.subplots(10,10,sharey=True,figsize=(40,40))
for N in range(10):
    n=10*N+4
    for i in range(9):
        axarr[N,i].imshow(X_compression_layers[i*10+1][n])
        axarr[N,i].axis('off')
        axarr[N,i].set_title('%i using %i dims' % (labels[n],10*i+1))
    axarr[N,9].imshow(images[n])
    axarr[N,9].axis('off')
    axarr[N,9].set_title('Original %i' %labels[n])
plt.savefig('PCA_digits_4.pdf')

In [ ]:
R=[np.sum(pca.explained_variance_ratio_[:i]) for i in range(150)]

In [ ]:
plt.plot(range(150),R,'bo')
plt.title("Explained variance with the first 100 Eigenimages")
plt.xlabel("Number of principal components")
plt.ylabel("Percent of variance explained")
plt.savefig('explained_variance.pdf')

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
pca2=PCA(n_components=3)
X_comp_3=pca2.fit_transform(val)

In [ ]:
fig=plt.figure()
ax=fig.add_subplot(111,projection='3d')
ax.scatter(X_comp_3[:,0],X_comp_3[:,1],X_comp_3[:,2],c=labels,cmap='tab20c')
plt.show()


In [ ]:
L=[]
depth=40
for i in range(1,depth):
    pcai=PCA(n_components=i)
    X_comp_i=pcai.fit_transform(val)
    X_test_i=pcai.transform(X_test)
    logreg=linear_model.LogisticRegression()
    logreg.fit(X_comp_i,labels)
    L.append(100*logreg.score(X_test_i,y_test))

In [ ]:
plt.plot(range(1,depth),L,'bo')
plt.title("Prediction accuracy of logistic classifier with n principal components")
plt.xlabel("n")
plt.ylabel("Prediction accuracy")
plt.ylim(0,100)
plt.xlim(0,depth)
plt.savefig('Predictions versus components.pdf')

In [ ]:
pca150=PCA(n_components=150)
X_comp_150=pca150.fit_transform(val)
X_test_150=pca150.transform(X_test)
logreg=linear_model.LogisticRegression()
logreg.fit(X_comp_150,labels)
print("Logistic Classifier prediction accuracy with 150 pc's: %s" 
      %(logreg.score(X_test_150,y_test)))

In [ ]:

#import ffmpeg

fig = plt.figure()


def f(x, y):
    return np.sin(x) + np.cos(y)

x = np.linspace(0, 2 * np.pi, 120)
y = np.linspace(0, 2 * np.pi, 100).reshape(-1, 1)

im = plt.imshow(f(x, y), animated=True)


def updatefig(*args):
    global x, y
    x += np.pi / 15.
    y += np.pi / 20.
    im.set_array(f(x, y))
    return im,
#Writer=animation.writers['ffmpeg']
#writer = Writer(fps=15, metadata=dict(artist='Me'), bitrate=1800)


ani = animation.FuncAnimation(fig, updatefig, interval=50, blit=True)
ani.save('pic')